In [1]:
import csv
from pathlib import Path

from cymruwhois import Client
#import ipresolved

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime, timedelta
from time import strftime
import pandas as pd
import re
import time
import requests
from dateutil import rrule
from time import mktime
from requests.exceptions import RetryError
from random import randint

from pytx import ThreatDescriptor
from pytx.vocabulary import ThreatExchange as te
from pytx.access_token import access_token
from pytx.logger import setup_logger
from pytx.vocabulary import PrivacyType as pt

ModuleNotFoundError: No module named 'cymruwhois'

In [2]:
# Load data from pre-made data.csv

with open('data.csv') as f:
    d = pd.read_csv(f, parse_dates=[],low_memory=False)
d

,Unnamed: 0,added_on,confidence,description,expired_on,first_active,id,indicator,last_active,last_updated,...,review_status,severity,share_level,source_uri,status,tags,type,ds,ts,name
0,0,2016-10-10T04:27:04+0000,50,Teaserinj Malicious Script Injection on Possib...,NaN,NaN,989335801175994,"{'id': '989335794509328', 'indicator': 'http:/...",NaN,2016-10-10T04:27:06+0000,...,UNREVIEWED,SEVERE,GREEN,NaN,MALICIOUS,NaN,URI,2016-10-10,2016-10-10 04:27:04,RiskIQ
1,0,2016-01-01T23:29:55+0000,90,Compromised domain in AS3741. Malicious URL:mo...,NaN,NaN,875325125915484,"{'id': '875325115915485', 'indicator': 'molgri...",NaN,2016-05-21T23:43:18+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:29:55,Cloudmark Threat Exchange
2,1,2016-01-01T23:24:49+0000,90,Compromised domain in AS32613. Malicious URL:p...,NaN,NaN,953897001325079,"{'id': '953896997991746', 'indicator': 'p13855...",NaN,2016-05-26T22:17:16+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:49,Cloudmark Threat Exchange
3,2,2016-01-01T23:24:48+0000,90,Compromised domain in AS32613. Malicious URL:h...,NaN,NaN,942876809114340,"{'id': '942876805781007', 'indicator': 'develo...",NaN,2016-05-29T00:08:32+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:48,Cloudmark Threat Exchange
4,3,2016-01-01T22:56:19+0000,90,Compromised domain in AS3223. Malicious URL:ww...,NaN,NaN,912399758852107,"{'id': '912399755518774', 'indicator': 'bateri...",NaN,2016-05-28T02:14:54+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:56:19,Cloudmark Threat Exchange
5,4,2016-01-01T22:49:12+0000,90,Compromised domain in AS6724. Malicious URL:ht...,NaN,NaN,930020557087476,"{'id': '930020553754143', 'indicator': 'predif...",NaN,2016-05-29T17:06:50+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:12,Cloudmark Threat Exchange
6,5,2016-01-01T22:49:11+0000,90,Compromised domain in AS20013. Malicious URL:i...,NaN,NaN,824054484369537,"{'id': '824054481036204', 'indicator': 'iprote...",NaN,2016-05-31T15:06:12+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:11,Cloudmark Threat Exchange
7,6,2016-01-01T22:48:10+0000,90,Compromised domain in AS46606. Malicious URL:h...,NaN,NaN,983145918419311,"{'id': '983145915085978', 'indicator': 'live--...",NaN,2016-05-19T12:49:11+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:48:10,Cloudmark Threat Exchange
8,7,2016-01-01T22:02:25+0000,90,Compromised domain in AS42355. Malicious URL:h...,NaN,NaN,946343195412806,"{'id': '946343192079473', 'indicator': 'kdev.d...",NaN,2016-05-27T10:34:31+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:02:25,Cloudmark Threat Exchange
9,8,2016-01-01T21:45:08+0000,90,Compromised domain in AS48505. Malicious URL:s...,NaN,NaN,1144064518967456,"{'id': '1144064515634123', 'indicator': 'slubn...",NaN,2016-05-26T05:05:23+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:45:08,Cloudmark Threat Exchange


In [3]:
d_filtered_compromised = d[d.description.str.startswith('Compromised', na=False)]
d_filtered_compromised

,Unnamed: 0,added_on,confidence,description,expired_on,first_active,id,indicator,last_active,last_updated,...,review_status,severity,share_level,source_uri,status,tags,type,ds,ts,name
1,0,2016-01-01T23:29:55+0000,90,Compromised domain in AS3741. Malicious URL:mo...,NaN,NaN,875325125915484,"{'id': '875325115915485', 'indicator': 'molgri...",NaN,2016-05-21T23:43:18+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:29:55,Cloudmark Threat Exchange
2,1,2016-01-01T23:24:49+0000,90,Compromised domain in AS32613. Malicious URL:p...,NaN,NaN,953897001325079,"{'id': '953896997991746', 'indicator': 'p13855...",NaN,2016-05-26T22:17:16+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:49,Cloudmark Threat Exchange
3,2,2016-01-01T23:24:48+0000,90,Compromised domain in AS32613. Malicious URL:h...,NaN,NaN,942876809114340,"{'id': '942876805781007', 'indicator': 'develo...",NaN,2016-05-29T00:08:32+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 23:24:48,Cloudmark Threat Exchange
4,3,2016-01-01T22:56:19+0000,90,Compromised domain in AS3223. Malicious URL:ww...,NaN,NaN,912399758852107,"{'id': '912399755518774', 'indicator': 'bateri...",NaN,2016-05-28T02:14:54+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:56:19,Cloudmark Threat Exchange
5,4,2016-01-01T22:49:12+0000,90,Compromised domain in AS6724. Malicious URL:ht...,NaN,NaN,930020557087476,"{'id': '930020553754143', 'indicator': 'predif...",NaN,2016-05-29T17:06:50+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:12,Cloudmark Threat Exchange
6,5,2016-01-01T22:49:11+0000,90,Compromised domain in AS20013. Malicious URL:i...,NaN,NaN,824054484369537,"{'id': '824054481036204', 'indicator': 'iprote...",NaN,2016-05-31T15:06:12+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:49:11,Cloudmark Threat Exchange
7,6,2016-01-01T22:48:10+0000,90,Compromised domain in AS46606. Malicious URL:h...,NaN,NaN,983145918419311,"{'id': '983145915085978', 'indicator': 'live--...",NaN,2016-05-19T12:49:11+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:48:10,Cloudmark Threat Exchange
8,7,2016-01-01T22:02:25+0000,90,Compromised domain in AS42355. Malicious URL:h...,NaN,NaN,946343195412806,"{'id': '946343192079473', 'indicator': 'kdev.d...",NaN,2016-05-27T10:34:31+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 22:02:25,Cloudmark Threat Exchange
9,8,2016-01-01T21:45:08+0000,90,Compromised domain in AS48505. Malicious URL:s...,NaN,NaN,1144064518967456,"{'id': '1144064515634123', 'indicator': 'slubn...",NaN,2016-05-26T05:05:23+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:45:08,Cloudmark Threat Exchange
10,9,2016-01-01T21:34:56+0000,90,Compromised domain in AS132145. Malicious URL:...,NaN,NaN,946455475422671,"{'id': '946455472089338', 'indicator': 'austra...",NaN,2016-05-25T05:20:40+0000,...,UNKNOWN,SUSPICIOUS,GREEN,NaN,SUSPICIOUS,NaN,DOMAIN,2016-01-01,2016-01-01 21:34:56,Cloudmark Threat Exchange


In [4]:
df = d_filtered_compromised.filter(items=['description', 'ts', 'ds'])
df

,description,ts,ds
1,Compromised domain in AS3741. Malicious URL:mo...,2016-01-01 23:29:55,2016-01-01
2,Compromised domain in AS32613. Malicious URL:p...,2016-01-01 23:24:49,2016-01-01
3,Compromised domain in AS32613. Malicious URL:h...,2016-01-01 23:24:48,2016-01-01
4,Compromised domain in AS3223. Malicious URL:ww...,2016-01-01 22:56:19,2016-01-01
5,Compromised domain in AS6724. Malicious URL:ht...,2016-01-01 22:49:12,2016-01-01
6,Compromised domain in AS20013. Malicious URL:i...,2016-01-01 22:49:11,2016-01-01
7,Compromised domain in AS46606. Malicious URL:h...,2016-01-01 22:48:10,2016-01-01
8,Compromised domain in AS42355. Malicious URL:h...,2016-01-01 22:02:25,2016-01-01
9,Compromised domain in AS48505. Malicious URL:s...,2016-01-01 21:45:08,2016-01-01
10,Compromised domain in AS132145. Malicious URL:...,2016-01-01 21:34:56,2016-01-01


In [5]:
ex_df = df.description.str.extract(r'(AS.\d*)')
#ex_df_20 = ex_df[0].value_counts()[:20]
ex_df_all = ex_df[0].value_counts()

ddata = pd.DataFrame()
ddata['as_name'] = ex_df_all.index.tolist()
ddata['as_counts'] = list(ex_df_all)
#ddata
#ddata.loc[(ddata.as_counts>100),['as_name','as_counts']]
ddata_100 = ddata.loc[(ddata.as_counts>100),['as_name','as_counts']]
ddata_100

,as_name,as_counts
0,AS26496,10604
1,AS46606,4083
2,AS20013,2627
3,AS16276,2109
4,AS24940,1365
5,AS8342,809
6,AS8560,791
7,AS36351,735
8,AS13335,723
9,AS32475,687


In [6]:
from bs4 import BeautifulSoup
import ipaddress

def get_ip_ranges(as_nr):
    base_url = 'https://ipinfo.io/'
    response = requests.get('https://ipinfo.io/{}'.format(as_nr))
    
    soup = BeautifulSoup(response.text, 'html.parser')
    
    as_s = []
    for link in soup.find_all('a'):
        aa = re.match(r'^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}.*', str(link.string).strip())
        if aa:
            as_s.append(aa.group(0))
    return as_s

hosts = []
for as_ in ddata_100['as_name']:
    ip_ranges = get_ip_ranges(as_)
    host_count = 0
    for ip_range in ip_ranges:
        add = ipaddress.ip_network(ip_range)
        host_count += len(list(add.hosts()))
    print('{}: {}'.format(as_, host_count))
    hosts.append(host_count)
print(hosts)

AS26496: 1853216
AS46606: 1076286
AS20013: 221102
AS16276: 2767148
AS24940: 1241498
AS8342: 354512
AS8560: 554410
AS36351: 4842514
AS13335: 2208044
AS32475: 447468
AS27715: 313744
AS12824: 205802
AS197695: 80530
AS25535: 38342
AS33070: 698694
AS34619: 29210
AS29182: 71466
AS9123: 28194
AS16509: 22837944
AS32392: 0
AS31815: 113092
AS31034: 302924
AS36024: 101510
AS25532: 79846
AS19994: 533862
AS32748: 163922
AS15967: 164042
AS198610: 15254
AS33182: 126472
AS14618: 11916118
AS12876: 577508
AS20738: 101230
AS32244: 250050
AS39134: 13292
AS6697: 1993944
AS198047: 29878
AS17447: 42252
AS13768: 1520378
AS43362: 8160
AS30968: 33016
AS0: 0
AS60781: 385184
AS31240: 6140
AS62082: 2546
AS201168: 0
AS393406: 0
AS32613: 263622
AS51559: 46228
AS6724: 195816
AS62282: 49834
AS15244: 40514
AS8972: 489118
AS37963: 13943266
AS47583: 35206
AS29550: 102866
AS198414: 10470
AS7506: 269612
AS51468: 7152
AS43333: 18926
AS20773: 293264
AS46475: 95452
AS26347: 195926
AS56067: 16838
AS51557: 8926
AS197712: 8180
A

In [7]:
ddata_100['possible_ip_counts'] = hosts
ddata_100

,as_name,as_counts,possible_ip_counts
0,AS26496,10604,1853216
1,AS46606,4083,1076286
2,AS20013,2627,221102
3,AS16276,2109,2767148
4,AS24940,1365,1241498
5,AS8342,809,354512
6,AS8560,791,554410
7,AS36351,735,4842514
8,AS13335,723,2208044
9,AS32475,687,447468


In [8]:
ratio = [c / ipc if ipc != 0 else 0 for c, ipc in zip(ddata_100['as_counts'], ddata_100['possible_ip_counts'])]
ddata_100['ratio'] = ratio
ddata_100

,as_name,as_counts,possible_ip_counts,ratio
0,AS26496,10604,1853216,0.005722
1,AS46606,4083,1076286,0.003794
2,AS20013,2627,221102,0.011881
3,AS16276,2109,2767148,0.000762
4,AS24940,1365,1241498,0.001099
5,AS8342,809,354512,0.002282
6,AS8560,791,554410,0.001427
7,AS36351,735,4842514,0.000152
8,AS13335,723,2208044,0.000327
9,AS32475,687,447468,0.001535


In [48]:

asn = ddata_100['as_name']
c=Client()
r=c.lookup(asn)
print(r)


IndexError: list index out of range

In [119]:
c=Client()
asn=ddata_100['as_name']

for i in asn:
    r=c.lookup(i)
    print('ASN: ',r)
#print('asn number: ',r.ip)
#print('asn owener : ',r.owner)
#print('==============')
#df1 = list_asn()

ASN:  26496      US 'AS-26496-GO-DADDY-COM-LLC - GoDaddy.com, LLC, US'
ASN:  46606      US 'UNIFIEDLAYER-AS-1 - Unified Layer, US'
ASN:  20013      US 'CYRUSONE - CyrusOne LLC, US'
ASN:  16276      FR 'OVH, FR'
ASN:  24940      DE 'HETZNER-AS, DE'
ASN:  8342       RU 'RTCOMM-AS, RU'
ASN:  8560       DE 'ONEANDONE-AS Brauerstrasse 48, DE'
ASN:  36351      US 'SOFTLAYER - SoftLayer Technologies Inc., US'
ASN:  13335      US 'CLOUDFLARENET - Cloudflare, Inc., US'
ASN:  32475      US 'SINGLEHOP-LLC - SingleHop LLC, US'
ASN:  27715      BR 'Locaweb Serviços de Internet S/A, BR'
ASN:  12824      PL 'HOMEPL-AS, PL'
ASN:  197695     RU 'AS-REG, RU'
ASN:  25535      RU 'ASN-RUCENTER-HOSTING, RU'
ASN:  33070      US 'RMH-14 - Rackspace Hosting, US'
ASN:  34619      TR 'CIZGI, TR'
ASN:  29182      LU 'ISPSYSTEM-AS, LU'
ASN:  9123       RU 'TIMEWEB-AS, RU'
ASN:  16509      US 'AMAZON-02 - Amazon.com, Inc., US'
ASN:  32392      US 'OPENTRANSFER-ECOMMERCE - The Endurance International Group, Inc., U